In [ ]:
!pip -q install google-generativeai newspaper3k

## Structured Output

In [ ]:
# setup
import google.generativeai as genai
from google.colab import userdata
import os

os.environ["GOOGLE_AI_STUDIO"] = userdata.get("GOOGLE_AI_STUDIO")

genai.configure(api_key=os.environ["GOOGLE_AI_STUDIO"])

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              # Set the `response_mime_type` to output JSON
                              generation_config={"response_mime_type": "application/json"})

prompt = """
  List 5 popular cookie recipes.
  Using this JSON schema:
    Recipe = {"recipe_name": str}
  Return a `list[Recipe]`
  """

response = model.generate_content(prompt)
print(response.text)

In [ ]:
type(response.text)

In [ ]:
import google.generativeai as genai
import typing_extensions as typing

class Recipe(typing.TypedDict):
  recipe_name: str
  ingredients: str


# Using `response_mime_type` with `response_schema` requires a Gemini 1.5 Pro model
model = genai.GenerativeModel("gemini-1.5-pro",
                              # Set the `response_mime_type` to output JSON
                              # Pass the schema object to the `response_schema` field
                              generation_config={"response_mime_type": "application/json",
                                                 "response_schema": list[Recipe]})

prompt = "List 5 popular cookie recipes"

response = model.generate_content(prompt)
print(response.text)

## Getting it into pydantic classes

In [ ]:
from pydantic import BaseModel
from typing import List
import json

# Define Pydantic models
class Recipe(BaseModel):
    ingredients: str
    recipe_name: str

class RecipeList(BaseModel):
    recipes: List[Recipe]


# Parse JSON data
data = json.loads(response.text)

# Create Pydantic model instance
recipe_list = RecipeList(recipes=[Recipe(**recipe) for recipe in data])

# Access and print recipe data
for recipe in recipe_list.recipes:
    print(f"Recipe: {recipe.recipe_name}")
    print(f"Ingredients: {recipe.ingredients}")
    print()

# Accessing a specific recipe
print("First recipe:")
print(recipe_list.recipes[0].recipe_name)
print(recipe_list.recipes[0].ingredients)

## Starting with Pydantic

In [ ]:
from enum import Enum
from pydantic import BaseModel

class ProductType(str, Enum):
    """
    Enumeration of product types.

    This enum represents different categories of products that can be referenced.
    """
    device = "device"
    app = "app"
    ft = "fintech"
    saas = "saas"
    consumer_tech = "consumer_tech"

class OrgType(str, Enum):
    """
    Enumeration of product types.

    This enum represents different categories of products that can be referenced.
    """
    startup = "startup"
    big_tech = "big_tech"
    tech_media = "tech_media"
    social_media = "social_media"
    gov = "govt_org"
    non_prof = "non_profit_org"
    vc = "venture_captial"
    other = "other"

class Person(BaseModel):
    """
    Represents an individual associated with an organization.

    This model captures basic information about a person, including their name,
    the organization they"re associated with, and their role within that organization.
    """
    name: str
    organization: str
    role: str

class Product(BaseModel):
    """
    Represents a product offered by an organization.

    This model captures basic information about a product, including its name,
    the organization it belongs to, and its type.
    """
    name: str
    organization: str
    product_type: ProductType

class Organization(BaseModel):
    """
    Represents an organization or company.

    This model captures basic information about an organization, including its name
    and location.
    """
    name: str
    org_type: OrgType
    location: str

class ArticleResponse(BaseModel):
    """
    Represents the structured response for an article analysis.

    This model aggregates information about products, people, and organizations
    mentioned in an article, along with a summary of the article"s content.
    """
    products: list[Product]
    people: list[Person]
    organizations: list[Organization]
    summary: str

In [ ]:
json_schema = ArticleResponse.schema_json(indent=2)
print(json_schema)

## Article getter

In [ ]:
import newspaper

def download_article(url):
    # Create an Article object
    article = newspaper.Article(url)

    # Download and parse the article
    article.download()
    article.parse()

    # Get the article text
    article_text = article.text

    return article_text


In [ ]:
url = "https://techcrunch.com/2024/08/13/made-by-google-2024-a-few-ai-features-you-mightve-missed/"
article = download_article(url)

# Now, "content" contains the text of the article
print("Article content:")
print(article[:500] + "...")

In [ ]:
import json

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction=f"""You are a helpful assistant that scans for
people, products and organizations mentioned in articles.
Using this JSON schema:
ArticleResponse = {json_schema}
Return a `ArticleResponse`""",
    generation_config={
        "response_mime_type": "application/json"
    }
)

response = model.generate_content(article)

json_object = json.loads(response.text)
print(json_object["people"])
for product in json_object["products"]:
    print(product)

print(json_object["organizations"])
print("Summary:",json_object["summary"])

In [ ]:
from pydantic import BaseModel
import json

class ArticleResponse(BaseModel):
    """
    Represents the structured response for an article analysis.

    This model aggregates information about products, people, and organizations
    mentioned in an article, along with a summary of the article's content.
    """
    products: list[Product]
    people: list[Person]
    organizations: list[Organization]
    summary: str

def parse_json_to_article_response(json_data: str) -> ArticleResponse:
    """
    Parse JSON data into an ArticleResponse object.

    :param json_data: JSON string containing article analysis data
    :return: ArticleResponse object
    """
    # Parse JSON string to dictionary
    data = json.loads(json_data) if isinstance(json_data, str) else json_data

    # Parse products
    products = [Product(
        name=p["name"],
        organization=p["organization"],
        product_type=ProductType(p["product_type"])
    ) for p in data["products"]]

    # Parse people
    people = [Person(
        name=p["name"],
        organization=p["organization"],
        role=p["role"]
    ) for p in data["people"]]

    # Parse organizations
    organizations = [Organization(
        name=o["name"],
        org_type=OrgType(o["org_type"]),
        location=o["location"]
    ) for o in data["organizations"]]

    # Create and return ArticleResponse
    return ArticleResponse(
        products=products,
        people=people,
        organizations=organizations,
        summary=data["summary"]
    )


# Parse JSON data
data = json.loads(response.text)

# Parse JSON to ArticleResponse
article_response = parse_json_to_article_response(data)

print(article_response)

In [ ]:
article_response.products

In [ ]:
article_response.summary

## With Images

In [ ]:
import PIL.Image

img = PIL.Image.open("timetable.png")
img

In [ ]:
import google.generativeai as genai
import typing_extensions as typing

class FlightInfo(typing.TypedDict):
  flight_time: str
  flight_destination: str


# Using `response_mime_type` with `response_schema` requires a Gemini 1.5 Pro model
model = genai.GenerativeModel("gemini-1.5-pro",
                              # Set the `response_mime_type` to output JSON
                              # Pass the schema object to the `response_schema` field
                              generation_config={"response_mime_type": "application/json",
                                                 "response_schema": list[FlightInfo]})

prompt = "List out the flight times and destinations"

response = model.generate_content([prompt,img])

print(response.text)

In [ ]:
for flight in json.loads(response.text):
  print(flight["flight_time"], flight["flight_destination"])

In [ ]:
import json

class FlightInfo(BaseModel):
    flight_time: str
    flight_destination: str

json_schema = FlightInfo.schema_json(indent=2)
print(json_schema)

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction=f"""You are a helpful assistant that scans for
flight times and destinations.
Using this JSON schema:
    FlightInfo = {json_schema}
Return a `list[FlightInfo]`""",
    generation_config={
        "response_mime_type": "application/json"
    }
)

prompt = "List out the flight times and destinations"

response = model.generate_content([prompt, img])

In [ ]:
response.text

In [ ]:
for flight in json.loads(response.text):
  print(flight["flight_time"], flight["flight_destination"])